In [54]:
import pandas as pd
import numpy as np

In [55]:
df = pd.read_csv("complex_median_weights.csv")

In [56]:
# Rename cols
df = df.rename(columns={"ASSEMBLY_STR": "asm_str", "PDB_COMPLEX_ID": "complex_id", "ASSEMBLY_NAME": "asm_name", "ASSEMBLY_TYPE": "asm_type",
                       "ASSEMBLIES": "assemblies", "NUM_ASSEMBLIES": "num_assemblies", "NUM_COMPONENTS": "num_components",
                       "MEDIAN_WT": "median_mw_kda", "MAX_WT": "max_mw_kda"})

In [57]:
df.head(5)

,asm_str,complex_id,asm_name,asm_type,assemblies,num_assemblies,num_components,median_mw_kda,max_mw_kda
0,A0A010_2,PDB-CPX-100020,MoeN5,homomeric,"5b00_1,5b00_2,5b01_1,5b01_2,5b01_3,5b01_4,5b01_5",7,2,64.52,65.18
1,A0A011_2,PDB-CPX-100021,MoeO5,homomeric,"3vk5_1,3vka_1,3vkb_1,3vkc_1,3vkd_1",5,2,63.15,63.56
2,A0A022MQ12_2,PDB-CPX-100028,Amidohydrolase-related domain-containing protein,homomeric,"6sj0_1,6sj1_1,6sj2_1,6sj3_1,6sj4_1",5,2,113.78,114.64
3,A0A022MRT4_2,PDB-CPX-100030,AMP-dependent synthetase/ligase domain-contain...,homomeric,"6six_1,6siy_1,6siz_1",3,2,98.65,98.84
4,A0A023DFE8_2,PDB-CPX-100031,Metallo-beta-lactamase domain-containing protein,homomeric,"6n9i_1,6n9i_2,6n9q_1,9ayt_1,9ayt_2,9b2i_1,9b2i...",16,2,70.40,71.83


In [58]:
# small_heteromers = df[
#     (df['asm_type'] == 'heteromeric') &
#     (df['num_assemblies'].between(10, 30)) &
#     (df['num_components'].between(3, 10))
# ]

# small_heteromers.to_csv("heteromeric_complexes.csv", index='False')

In [59]:
# Ensure numeric
df["num_assemblies"] = pd.to_numeric(df["num_assemblies"], errors="coerce")
df["median_mw_kda"] = pd.to_numeric(df["median_mw_kda"], errors="coerce")

In [60]:
# Drop unusable rows
df = df.dropna(subset=["complex_id", "num_assemblies", "median_mw_kda"]).copy()

In [61]:
# Add bin labels (A_BIN and MW_BIN)

a_edges = [3, 5, 9, 21, 101, np.inf]  # [3-4], [5-8], [9-20], [21-100], [>=101]
a_labels = ["A1_3-4", "A2_5-8", "A3_9-20", "A4_21-100", "A5_101+"]

df["A_BIN"] = pd.cut(
    df["num_assemblies"],
    bins=a_edges,
    right=False,        # include left edge, exclude right edge
    labels=a_labels
)

mw_edges = [0, 60, 120, 300, np.inf]
mw_labels = ["MW1_<60", "MW2_60-120", "MW3_120-300", "MW4_>300"]

df["MW_BIN"] = pd.cut(
    df["median_mw_kda"],
    bins=mw_edges,
    right=False,
    labels=mw_labels
)

In [62]:
df.head(5)

,asm_str,complex_id,asm_name,asm_type,assemblies,num_assemblies,num_components,median_mw_kda,max_mw_kda,A_BIN,MW_BIN
0,A0A010_2,PDB-CPX-100020,MoeN5,homomeric,"5b00_1,5b00_2,5b01_1,5b01_2,5b01_3,5b01_4,5b01_5",7,2,64.52,65.18,A2_5-8,MW2_60-120
1,A0A011_2,PDB-CPX-100021,MoeO5,homomeric,"3vk5_1,3vka_1,3vkb_1,3vkc_1,3vkd_1",5,2,63.15,63.56,A2_5-8,MW2_60-120
2,A0A022MQ12_2,PDB-CPX-100028,Amidohydrolase-related domain-containing protein,homomeric,"6sj0_1,6sj1_1,6sj2_1,6sj3_1,6sj4_1",5,2,113.78,114.64,A2_5-8,MW2_60-120
3,A0A022MRT4_2,PDB-CPX-100030,AMP-dependent synthetase/ligase domain-contain...,homomeric,"6six_1,6siy_1,6siz_1",3,2,98.65,98.84,A1_3-4,MW2_60-120
4,A0A023DFE8_2,PDB-CPX-100031,Metallo-beta-lactamase domain-containing protein,homomeric,"6n9i_1,6n9i_2,6n9q_1,9ayt_1,9ayt_2,9b2i_1,9b2i...",16,2,70.40,71.83,A3_9-20,MW2_60-120


In [63]:
# Save "complexes_with_bins.csv" (the main formatted table)
out_with_bins = "complexes_with_bins.csv"
df_out = df[["complex_id", "num_assemblies", "median_mw_kda", "A_BIN", "MW_BIN"]].copy()
df_out.to_csv(out_with_bins, index=False)

In [64]:
df_out.head(5)

,complex_id,num_assemblies,median_mw_kda,A_BIN,MW_BIN
0,PDB-CPX-100020,7,64.52,A2_5-8,MW2_60-120
1,PDB-CPX-100021,5,63.15,A2_5-8,MW2_60-120
2,PDB-CPX-100028,5,113.78,A2_5-8,MW2_60-120
3,PDB-CPX-100030,3,98.65,A1_3-4,MW2_60-120
4,PDB-CPX-100031,16,70.40,A3_9-20,MW2_60-120


In [65]:
# Create cell counts table (how many complexes per bin cell)
cell_counts = (
    df_out.groupby(["A_BIN", "MW_BIN"])
          .size()
          .reset_index(name="n_complexes")
          .sort_values(["A_BIN", "MW_BIN"])
)
cell_counts.to_csv("cell_counts.csv", index=False)

/var/folders/qn/p72f2jb124x0vh2scsqtln940000gp/T/ipykernel_41701/3761554288.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_out.groupby(["A_BIN", "MW_BIN"])


In [66]:
cell_counts

,A_BIN,MW_BIN,n_complexes
0,A1_3-4,MW1_<60,2688
1,A1_3-4,MW2_60-120,2223
2,A1_3-4,MW3_120-300,1435
3,A1_3-4,MW4_>300,639
4,A2_5-8,MW1_<60,1470
5,A2_5-8,MW2_60-120,1340
6,A2_5-8,MW3_120-300,840
7,A2_5-8,MW4_>300,333
8,A3_9-20,MW1_<60,701
9,A3_9-20,MW2_60-120,669


In [67]:
# Choose the "median complex" in each (A_BIN, MW_BIN) cell
# Definition: sort by median_mw_kda, pick the middle row (upper median)
def pick_median_complex(group: pd.DataFrame) -> pd.DataFrame:
    g = group.sort_values("median_mw_kda", ascending=True).reset_index(drop=True)
    idx = len(g) // 2   # upper median for even n
    return g.iloc[[idx]]

median_complexes = (
    df_out.groupby(["A_BIN", "MW_BIN"], group_keys=False)
          .apply(pick_median_complex)
          .reset_index(drop=True)
)

median_complexes.to_csv("phase1_median_complexes.csv", index=False)

/var/folders/qn/p72f2jb124x0vh2scsqtln940000gp/T/ipykernel_41701/2058041943.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_out.groupby(["A_BIN", "MW_BIN"], group_keys=False)
/var/folders/qn/p72f2jb124x0vh2scsqtln940000gp/T/ipykernel_41701/2058041943.py:10: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(pick_median_complex)


In [68]:
median_complexes["complex_id"].to_csv("complex_ids.txt", index=False, header=False)

In [69]:
# Read the benchmark run dataset
bd = pd.read_csv("benchmark_dataset.csv")

In [70]:
# Add number of pairs column
df_out["num_pairs"] = (
    df_out["num_assemblies"] *
    (df_out["num_assemblies"] - 1)
) // 2

In [71]:
# Merge with timing table
df_combined = df_out.merge(
    bd,
    on=["A_BIN", "MW_BIN"],
    how="left"
)

In [72]:
# Only include columns needed for the final df
df_final = df_combined[['complex_id_x', 'num_assemblies_x', 'median_mw_kda', 'A_BIN', 'MW_BIN', 'num_pairs_x', 'avg_time_sec']]

In [73]:
df_final.head(5)

,complex_id_x,num_assemblies_x,median_mw_kda,A_BIN,MW_BIN,num_pairs_x,avg_time_sec
0,PDB-CPX-100020,7,64.52,A2_5-8,MW2_60-120,21,31.68
1,PDB-CPX-100021,5,63.15,A2_5-8,MW2_60-120,10,31.68
2,PDB-CPX-100028,5,113.78,A2_5-8,MW2_60-120,10,31.68
3,PDB-CPX-100030,3,98.65,A1_3-4,MW2_60-120,3,33.22
4,PDB-CPX-100031,16,70.40,A3_9-20,MW2_60-120,120,28.27


In [74]:
# Rename selected columns. Some columns appear twice in the orginal merged dfs
df_final = df_final.rename(columns={
    "complex_id_x": "complex_id",
    "num_assemblies_x": "num_assemblies",
    "num_pairs_x": "num_pairs"
})

In [75]:
# Calculate serial wall hours
df_final["est_total_wall_hours"] = (
    df_final["num_pairs"] * df_final["avg_time_sec"]
 / 3600)

In [76]:
df_final.head(5)

,complex_id,num_assemblies,median_mw_kda,A_BIN,MW_BIN,num_pairs,avg_time_sec,est_total_wall_hours
0,PDB-CPX-100020,7,64.52,A2_5-8,MW2_60-120,21,31.68,0.184800
1,PDB-CPX-100021,5,63.15,A2_5-8,MW2_60-120,10,31.68,0.088000
2,PDB-CPX-100028,5,113.78,A2_5-8,MW2_60-120,10,31.68,0.088000
3,PDB-CPX-100030,3,98.65,A1_3-4,MW2_60-120,3,33.22,0.027683
4,PDB-CPX-100031,16,70.40,A3_9-20,MW2_60-120,120,28.27,0.942333


In [77]:
# Calculate parallel wall hours
k = 100
df_final["parallel_wall_hours_k100"] = df_final["est_total_wall_hours"] / k

In [78]:
df_final.head(10)

,complex_id,num_assemblies,median_mw_kda,A_BIN,MW_BIN,num_pairs,avg_time_sec,est_total_wall_hours,parallel_wall_hours_k100
0,PDB-CPX-100020,7,64.52,A2_5-8,MW2_60-120,21,31.68,0.184800,0.001848
1,PDB-CPX-100021,5,63.15,A2_5-8,MW2_60-120,10,31.68,0.088000,0.000880
2,PDB-CPX-100028,5,113.78,A2_5-8,MW2_60-120,10,31.68,0.088000,0.000880
3,PDB-CPX-100030,3,98.65,A1_3-4,MW2_60-120,3,33.22,0.027683,0.000277
4,PDB-CPX-100031,16,70.40,A3_9-20,MW2_60-120,120,28.27,0.942333,0.009423
5,PDB-CPX-100035,4,18.71,A1_3-4,MW1_<60,6,19.43,0.032383,0.000324
6,PDB-CPX-100040,4,104.87,A1_3-4,MW2_60-120,6,33.22,0.055367,0.000554
7,PDB-CPX-100059,3,245.40,A1_3-4,MW3_120-300,3,66.39,0.055325,0.000553
8,PDB-CPX-100064,17,221.55,A3_9-20,MW3_120-300,136,94.73,3.578689,0.035787
9,PDB-CPX-100068,5,19.54,A2_5-8,MW1_<60,10,14.09,0.039139,0.000391


In [79]:
# Total serial hours to run this across the PDB archive
total_serial_hours = df_final["est_total_wall_hours"].sum()
print(f"Total serial hours: {total_serial_hours:.2f}")

Total serial hours: 107841.82


In [80]:
# Total parallel hours to run this across the PDB archive
total_parallel_hours = df_final["parallel_wall_hours_k100"].sum()
print(f"Total parallel hours: {total_parallel_hours:.2f}")

Total parallel hours: 1078.42


In [81]:
# Total serial and parallel days to run this across the PDB archive
total_serial_days = total_serial_hours / 24
total_parallel_days = total_parallel_hours / 24

print(f"Total serial days: {total_serial_days:.2f}")
print(f"Total parallel days (k=100): {total_parallel_days:.2f}")

Total serial days: 4493.41
Total parallel days (k=100): 44.93


In [82]:
df_final.to_csv('resource_estimate.csv', index=False)

In [83]:
# Group by bins combination
by_bin = (
    df_final.groupby(["A_BIN", "MW_BIN"], as_index=False)
    .agg(
        num_complexes=("complex_id", "nunique"),
        serial_hours=("est_total_wall_hours", "sum"),
        parallel_hours_k100=("parallel_wall_hours_k100", "sum"),
    )
)

serial_total = by_bin["serial_hours"].sum()
parallel_total = by_bin["parallel_hours_k100"].sum()

by_bin["pct_serial"] = (by_bin["serial_hours"] / serial_total * 100)
by_bin["pct_parallel"] = (by_bin["parallel_hours_k100"] / parallel_total * 100)

# Round for display
by_bin["serial_hours"] = by_bin["serial_hours"].round(2)
by_bin["parallel_hours_k100"] = by_bin["parallel_hours_k100"].round(2)
by_bin["pct_serial"] = by_bin["pct_serial"].round(2)
by_bin["pct_parallel"] = by_bin["pct_parallel"].round(2)

by_bin = by_bin.sort_values("serial_hours", ascending=False).reset_index(drop=True)

In [84]:
by_bin

,A_BIN,MW_BIN,num_complexes,serial_hours,parallel_hours_k100,pct_serial,pct_parallel
0,A5_101+,MW4_>300,5,74601.99,746.02,69.18,69.18
1,A5_101+,MW2_60-120,23,17522.54,175.23,16.25,16.25
2,A5_101+,MW1_<60,24,2934.64,29.35,2.72,2.72
3,A4_21-100,MW3_120-300,171,2629.37,26.29,2.44,2.44
4,A4_21-100,MW4_>300,63,2591.16,25.91,2.40,2.40
5,A4_21-100,MW2_60-120,245,1960.72,19.61,1.82,1.82
6,A4_21-100,MW1_<60,256,1120.24,11.20,1.04,1.04
7,A3_9-20,MW3_120-300,523,1066.45,10.66,0.99,0.99
8,A3_9-20,MW4_>300,191,856.02,8.56,0.79,0.79
9,A2_5-8,MW4_>300,333,610.24,6.10,0.57,0.57


In [85]:
by_bin.to_csv("resorce_estimation_grouped_by_categories.csv", index=False)